<h1> 10. Null model for analysis 1 </h1>

Run analysis 1 for networks generated from random seed words. Very similar steps as in 7. Analysis 1 (Co-occurence networks).ipynb

In [27]:
#imports

#functions
import pandas as pd
import pickle
import numpy as np
import math
import networkx as nx
import matplotlib.pyplot as plt
from community import community_louvain



#time_words
ntp_words = ['time', 'period', 'periods', 'duration', 'clock', 'temporal', 'spacetime', 'timespan', 'timespans', 'timeline', 'timelines', 'elapse', 'elapsed', 'length', 'timewise', 'velocity', 'pace', 'rate', 'tempo', 'pass', 'passing', 'passed']
ftp_words = ['quick','quicker', 'quickly', 'quickest', 'fast', 'faster', 'fastest', 'fastened', 'rapid','rapidly', 'short', 'shorter', 'shortly', 'shortest','speedy', 'speedy','speeded', 'speedier', 'hurry', 'hurried', 'swift', 'swifter', 'swiftly', 'haste', 'hasty', 'brisk', 'turbo', 'accelerate', 'acceleration', 'accelerated', 'accelerating']
stp_words = ['slow', 'slower', 'slowly', 'slows', 'slowed', 'slowest', 'slowing', 'slowdown', 'long', 'looong', 'longer', 'longer', 'longest', 'steady', 'deceleration', 'decelerate', 'decelerating', 'decelerated', 'dilatory', 'dilation', 'infinity', 'eternity', 'lengthy', 'prolonged', 'protracted', 'extended', 'unending', 'endless']
time_words = sorted(ntp_words + ftp_words + stp_words)


In [8]:
def aggregate_for_class(df2, class_): #"class_" can refer both class or substance with many reports

    #exclude words of random list from networks
    random_list = list(df2.seed.unique()) # remove random list 
    df2 = df2[~df2.filter(items=['source', 'target']).isin(random_list).any(axis=1)]


    df2.insert(4, 'weight', 1)

    if class_ == "all":
        #remove non class rows
        grouped = df2.groupby(["classes", "source", "target"], as_index=False)
        
        
    elif class_ in ["Serotonergic psychedelics", "Dissociative psychedelics", "Entactogens", "Deliriants", "Depressant / sedatives", "Stimulants", "Antidepressants / antipsychotics"]:
        #remove non class rows
        df2 = df2[df2['classes'] == class_]
        grouped = df2.groupby(["source", "target"], as_index=False)


    elif class_ in ["LSD", "Psilocybin mushrooms", "DMT", "MDMA", "Cannabis spp.", "Salvia divinorum"]:
        #remove non substance rows
        df2 = df2[df2['substance'] == class_.lower()]
        grouped = df2.groupby(["source", "target"], as_index=False)
    

    df2 = grouped[["weight"]].agg({'weight': np.sum}) 


    #formatting
    df2.sort_values(by=["weight"], ascending=False, inplace=True)
    df2.reset_index(drop=True, inplace=True)  

    return df2

In [9]:
def exclude_words(df2):


    #exclude self-loops (source = target)
    df2 = df2[df2['source'] != df2['target']]

    #remove non_seed_time_words + ... - 
    frequent_non_seed_time_words = ["second", "seconds", "minute", "minutes", "hour", "hours", "day", "days", "week", "weeks", "weekend", "weekends", "month", "months", "year", "years", "times", "spend", "spent", "spending", "timestamp", "timestamps"]

    df2 = df2[~df2.filter(items=['source', 'target']).isin(frequent_non_seed_time_words).any(axis=1)]



    return df2

In [10]:
def filter_tfidf(tfidf_df, class_, df2, gamma):  

    if class_ in ["LSD", "Psilocybin mushrooms", "DMT", "MDMA", "Cannabis spp.", "Salvia divinorum"]:
        class_ = class_.lower()
    
    #sort values by highest to lowest tfidf scores for a class
    tfidf_df.sort_values(by=class_, ascending=False, inplace=True)
    tfidf_df.reset_index(drop=True, inplace=True)


    #select top delta tfidf words and filter all edges where both words not in top tfidf
    top_tfidf = tfidf_df["word"].to_list()[:gamma]
    df2 = df2[df2['source'].isin(top_tfidf) & df2['target'].isin(top_tfidf)]


    #formatting
    df2.sort_values(by=["weight"], ascending=False, inplace=True)
    df2.reset_index(drop=True, inplace=True)

    return df2

In [11]:
def filterby_rw(df2, delta):
    
    #create temporary graph G1 to calculate strength and degree for each node
    G1 = nx.from_pandas_edgelist(df2, edge_attr="weight")
    strength_dict = dict(G1.degree(weight="weight"))
    degree_dict = dict(G1.degree())

    df2["prob_source"] = 0
    df2["prob_target"] = 0

    #iterate through all rows and calculate prob of null model for each edge
    for i, weight in enumerate(df2.weight):
        source = df2.loc[i, "source"]
        target = df2.loc[i, "target"]

        df2.loc[i, "prob_source"] = (1 - (weight / strength_dict[source]))**(degree_dict[source]-1)
        df2.loc[i, "prob_target"] = (1 - (weight / strength_dict[target]))**(degree_dict[target]-1)

    
    #delta threshold for null model -> filter all edges where the prob of either node for null model is above delta
    df2 = df2[((df2["prob_source"] < delta) & (df2["prob_target"] < delta))]
    df2.drop(["prob_source", "prob_target"], axis=1, inplace=True)
    df2.reset_index(drop=True, inplace=True)


    #formating
    df2.sort_values(by=["weight"], ascending=False, inplace=True)
    df2.reset_index(drop=True, inplace=True)

    G2 = nx.from_pandas_edgelist(df2, edge_attr="weight")


    return df2, G2

<h4> Modularity scores from random networks </h4>

Repeat steps as in analysis 1. Instead of creating GEFX file, we calculate the Louvian modularity for each random network.

In [ ]:
#data and tfidf scores
df2 = pd.read_pickle("timecorpus_C=4.pkl")
tfidf_df = pd.read_pickle("tfidf_df_C=4.pkl")
tfidf_df_SUB = pd.read_pickle("tfidf_df_C=4_SUBSTANCES.pkl")
tfidf_df_merged = pd.merge(tfidf_df, tfidf_df_SUB, on='word')

modularity_dict = {}

for j in range(15):
    print(j)
    df2 = pd.read_pickle(f"Random corpus\_randomcorpus_C=4_j={j}.pkl")
    def run_class(df2, class_, tfidf_df, gamma, delta):
        df2 = aggregate_for_class (df2, class_)
        df2 = exclude_words(df2)
        df2 = filter_tfidf(tfidf_df, class_, df2, gamma)
        df2, G2 = filterby_rw(df2, delta)
        return df2, G2

    #Choose class_, select gamma, delta with same values for each class as you have 
    #all Serotonergic psychedelics, Dissociative psychedelics, Entactogens, Deliriants, Depressant / sedatives, Stimulants, Antidepressants / antipsychotics, LSD, Psilocybin mushrooms, DMT, MDMA, Cannabis spp., Salvia divinorum
    df2, G2  = run_class(df2=df2, class_="Serotonergic psychedelics", tfidf_df=tfidf_df_merged, gamma=5000, delta=0.03)

    partition = nx.community.louvain_communities(G2, weight='weight')

    # Compute the modularity score
    modularity = nx.algorithms.community.modularity(G2, partition)
    modularity_dict[j] = [modularity, len(df2), partition]

<h4> Modularity by violin plot & index  </h4>

In [ ]:

#Get list of modularity scores
modularity_scores = [modularity_dict[value][0] for value in modularity_dict]


# Modularity score when done with time perception seed words 
specific_score = 0.7 #

#add to modularity scores
modularity_scores.append(specific_score)


# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

# Create the violin plot on the left subplot
ax1.violinplot(modularity_scores, showmeans=True)

# Add a title and axis labels
ax1.set_title('Modularity Scores')
ax1.set_xlabel('Distribution')
ax1.set_ylabel('Score')

# Create the dot plot on the right subplot
positions = np.arange(len(modularity_scores))
ax2.scatter(positions, modularity_scores, color='blue', alpha=0.5, label='Random seed words')

# Add the specific score as a red dot
ax2.scatter(positions[modularity_scores.index(specific_score)], specific_score, color='red', label='Time perception seed words')

# Add an arrow with a label pointing at the specific score
ax2.annotate('Specific score', xy=(positions[modularity_scores.index(specific_score)], specific_score), xytext=(positions[modularity_scores.index(specific_score)]+3, specific_score+0.1), arrowprops=dict(facecolor='black', arrowstyle='->'))

# Add a legend
ax2.legend()

# Add a title and axis labels
ax2.set_title('Modularity Scores with Specific Score')
ax2.set_xlabel('Index')
ax2.set_ylabel('Score')

# Show the plot
plt.show()
